# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [36]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf_1(a, b, c): # TODO
  df_da = -3*(a**2) - 0.5*(a**(-0.5))
  df_db = 3*cos(3*b) + 2.5*b**1.5
  df_dc = 1/(c**2)
  return [df_da, df_db, df_dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf_1(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [40]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
def gradf_2(a, b, c):
  h = 1e-3
  df_da = (f(a+h, b, c)-f(a, b, c))/h
  df_db = (f(a, b+h, c)-f(a, b, c))/h
  df_dc = (f(a, b, c+h)-f(a, b, c))/h

  return [df_da, df_db, df_dc] # TODO

numerical_grad = gradf_2(2, 3, 4)
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")
  print('approx_ :', numerical_grad[dim] - ans[dim])


WRONG! for dim 0: expected -12.353553390593273, yours returns -12.359510207462066
approx_ : -0.005956816868792458
WRONG! for dim 1: expected 10.25699027111255, yours returns 10.2583876150808
approx_ : 0.0013973439682501976
WRONG! for dim 2: expected 0.0625, yours returns 0.06248437890565839
approx_ : -1.5621094341611297e-05


In [39]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
def gradf_3(a, b, c):
  h = 1e-3
  df_da = (f(a+h, b, c)-f(a-h, b, c))/(2*h)
  df_db = (f(a, b+h, c)-f(a, b-h, c))/(2*h)
  df_dc = (f(a, b, c+h)-f(a, b, c-h))/(2*h)

  return [df_da, df_db, df_dc] # TODO

numerical_grad2 = gradf_3(2, 3, 4) # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")
  print('approx_ :', numerical_grad[dim] - ans[dim])


#####
## If you see the approx_ results between two cells for h=1e-3,
## you see that we have lower error in second version of approximation

OK for dim 0: expected -12.353553390593273, yours returns -12.353554401639766
approx_ : -5.958216721779763e-06
OK for dim 1: expected 10.25699027111255, yours returns 10.256994551617105
approx_ : 1.3955669331267018e-06
OK for dim 2: expected 0.0625, yours returns 0.06250000390650712
approx_ : -1.5256830465659732e-08


## section 2: support for softmax

In [52]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  # ------
  # re-implement all the other functions needed for the exercises below
  def exp(self):
    # return math.exp(self.data)
    t = math.exp(self.data)
    return Value(t, (self, ), 'exp')
  # your code here
  # TODO
  # ------

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [43]:
import math
math.exp(1)

2.718281828459045

In [54]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
print(Value(0.0))
print(logits)
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


Value(data=0.0)
[Value(data=0.0), Value(data=3.0), Value(data=-2.0), Value(data=1.0)]
Value(data=1.0)


TypeError: unsupported operand type(s) for +: 'int' and 'Value'

In [65]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

x1 = torch.Tensor([0.0]).double();    x1.requires_grad = True
x2 = torch.Tensor([3.0]).double();    x2.requires_grad = True
x3 = torch.Tensor([-2.0]).double();    x3.requires_grad = True
w1 = torch.Tensor([0.5]).double();   w1.requires_grad = True
w2 = torch.Tensor([1.0]).double();    w2.requires_grad = True
w3 = torch.Tensor([1.0]).double();    w3.requires_grad = True

b = torch.Tensor([0.45465]).double();    b.requires_grad = True
f = w1*x1 + w2*x2 + w3*x3 + b
o = torch.softmax(f, dim=0)

print(o.data.item())
o.backward()

print('x1', x1.grad.item())
print('w1', w1.grad.item())
print('x2', x2.grad.item())
print('w2', w2.grad.item())

1.0
x1 0.0
w1 0.0
x2 0.0
w2 0.0


In [ ]:
torch